# FineRec preprocess 2

In [8]:
import os
import pickle
import time
import pandas as pd
import numpy as np

meta_dataset = 'Yelp'
data_path =  './dict/' + meta_dataset + '/session_data.csv'

if meta_dataset == 'Beauty':
    # 'Brand','Ingredients','Effectiveness','Scent','Price','Color','Size'
    attr_list = ['Brand','Ingredients','Effectiveness','Scent','Price','Color','Size', 'Compatibility', 'Longevity']
elif meta_dataset == 'Cell_Phones_and_Accessories':
    # Brand, Performance, color, Battery ,Connectivity,Price, Size
    attr_list = ['Brand', 'Performance', 'Color', 'Battery' ,'Connectivity','Price', 'Size']
elif meta_dataset == 'Sports_and_Outdoors':
    # Price, Quality, Functionality,Size,Protection,Material,Comfort
    attr_list = ['Price', 'Quality', 'Functionality', 'Size', 'Protection', 'Material', 'Comfort']
elif meta_dataset == 'Yelp':
    # Price, Brand, Quality, Material, Size, Appearance, Washability, Functionality, Style, Durability,
    attr_list = ['Price', 'Food', 'Location', 'Ambience', 'Service', 'Cleanliness', 'Parking']

data_all = pd.read_csv(data_path)
if meta_dataset == 'Beauty' or meta_dataset == 'Home_and_Kitchen':
    data_all = data_all.reset_index()[['sessionID', 'itemID', attr_list[0], attr_list[1],attr_list[2],attr_list[3],attr_list[4],attr_list[5],attr_list[6],attr_list[7],attr_list[8]]]
else:
    data_all = data_all.reset_index()[['sessionID', 'itemID', attr_list[0], attr_list[1],attr_list[2],attr_list[3],attr_list[4],attr_list[5],attr_list[6]]]

attr_num = len(attr_list)

# recorde all items
# dict {sessionID:[itemID,itemID]} 
sess_all = {}
#  dict {sessionID:[opinID,opinID]}
attr1_all = {}
attr2_all = {}
attr3_all = {}
attr4_all = {}
attr5_all = {}
attr6_all = {}
attr7_all = {}
attr8_all = {}
attr9_all = {}

item_count = 0
user_count = 0
inter_count = 0

for _, row in data_all.iterrows():
    sess_id = row['sessionID']
    item_id = row['itemID']
    attr1_opin = row[attr_list[0]]
    attr2_opin = row[attr_list[1]]
    attr3_opin = row[attr_list[2]]
    attr4_opin = row[attr_list[3]]
    attr5_opin = row[attr_list[4]]
    attr6_opin = row[attr_list[5]]
    attr7_opin = row[attr_list[6]]
    if meta_dataset == 'Beauty' or meta_dataset == 'Home_and_Kitchen':
        attr8_opin = row[attr_list[7]]
        attr9_opin = row[attr_list[8]]
    if sess_id in sess_all:
        sess_all[sess_id].append(item_id)
        attr1_all[sess_id].append(attr1_opin)
        attr2_all[sess_id].append(attr2_opin)
        attr3_all[sess_id].append(attr3_opin)
        attr4_all[sess_id].append(attr4_opin)
        attr5_all[sess_id].append(attr5_opin)
        attr6_all[sess_id].append(attr6_opin)
        attr7_all[sess_id].append(attr7_opin)
        if meta_dataset == 'Beauty' or meta_dataset == 'Home_and_Kitchen':
            attr8_all[sess_id].append(attr8_opin)
            attr9_all[sess_id].append(attr9_opin)
    else:
        sess_all[sess_id] = []
        sess_all[sess_id].append(item_id)
        attr1_all[sess_id] = []
        attr1_all[sess_id].append(attr1_opin)
        attr2_all[sess_id] = []
        attr2_all[sess_id].append(attr2_opin)
        attr3_all[sess_id] = []
        attr3_all[sess_id].append(attr3_opin)
        attr4_all[sess_id] = []
        attr4_all[sess_id].append(attr4_opin)
        attr5_all[sess_id] = []
        attr5_all[sess_id].append(attr5_opin)
        attr6_all[sess_id] = []
        attr6_all[sess_id].append(attr6_opin)
        attr7_all[sess_id] = []
        attr7_all[sess_id].append(attr7_opin)
        if meta_dataset == 'Beauty' or meta_dataset == 'Home_and_Kitchen':
            attr8_all[sess_id] = []
            attr9_all[sess_id] = []
            attr8_all[sess_id].append(attr8_opin)
            attr9_all[sess_id].append(attr9_opin)

# 限制每个seq长度
for sess_temp in sess_all.keys():
    seq = sess_all[sess_temp]
    if len(seq) >20:
        sess_all[sess_temp] = seq[-20:]
        attr1_all[sess_id] = attr1_all[sess_id][-20:]
        attr2_all[sess_id] = attr2_all[sess_id][-20:]
        attr3_all[sess_id] = attr3_all[sess_id][-20:]
        attr4_all[sess_id] = attr4_all[sess_id][-20:]
        attr5_all[sess_id] = attr5_all[sess_id][-20:]
        attr6_all[sess_id] = attr6_all[sess_id][-20:]
        attr7_all[sess_id] = attr7_all[sess_id][-20:]
        if meta_dataset == 'Beauty' or meta_dataset == 'Home_and_Kitchen':
            attr8_all[sess_id] = attr8_all[sess_id][-20:]
            attr9_all[sess_id] = attr9_all[sess_id][-20:]

# 形成train set
# 统计 train set 出现的item，给item重新编号
item_num = 1
# dict{lod_itemID:new_itemID}
item_dict = {}
# train_set [[u_id ...], [x_t-1...], [label...]
tra = [[],[],[]]
# dict {train_sessionID:[itemID,itemID]}  去掉了seq中last item(不包含test的label)， 也可以调整为去掉两个item
train_sess = {}

# dict {item_id:[user_id, user_id]}
train_items = {}

# dict {item_id:opin_id}
attr1_item_opin = {}
attr2_item_opin = {}
attr3_item_opin = {}
attr4_item_opin = {}
attr5_item_opin = {}
attr6_item_opin = {}
attr7_item_opin = {}
attr8_item_opin = {}
attr9_item_opin = {}


for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    train_seqs_t = all_seqs[:-1]
    a1_opin = attr1_all[sess_temp][:-1]
    a2_opin = attr2_all[sess_temp][:-1]
    a3_opin = attr3_all[sess_temp][:-1]
    a4_opin = attr4_all[sess_temp][:-1]
    a5_opin = attr5_all[sess_temp][:-1]
    a6_opin = attr6_all[sess_temp][:-1]
    a7_opin = attr7_all[sess_temp][:-1]
    if meta_dataset == 'Beauty' or meta_dataset == 'Home_and_Kitchen':
        a8_opin = attr8_all[sess_temp][:-1]
        a9_opin = attr9_all[sess_temp][:-1]
    else:
        a8_opin = attr6_all[sess_temp][:-1]
        a9_opin = attr7_all[sess_temp][:-1]
    inter_count += len(train_seqs_t)
#     item_seq with new ID
    new_seq = []
    for x, a1, a2, a3, a4, a5, a6, a7, a8, a9 in zip(train_seqs_t, a1_opin, a2_opin,a3_opin,a4_opin,a5_opin,a6_opin,a7_opin,a8_opin,a9_opin):
        if x in item_dict:
            new_seq.append(item_dict[x])
            if sess_temp not in train_items[item_dict[x]]:
                train_items[item_dict[x]].append(sess_temp)
                attr1_item_opin[item_dict[x]].append(a1)
                attr2_item_opin[item_dict[x]].append(a2)
                attr3_item_opin[item_dict[x]].append(a3)
                attr4_item_opin[item_dict[x]].append(a4)
                attr5_item_opin[item_dict[x]].append(a5)
                attr6_item_opin[item_dict[x]].append(a6)
                attr7_item_opin[item_dict[x]].append(a7)
                if meta_dataset == 'Beauty' or meta_dataset == 'Home_and_Kitchen':
                    attr8_item_opin[item_dict[x]].append(a8)
                    attr9_item_opin[item_dict[x]].append(a9)
        else:
            item_dict[x] = item_num
            new_seq.append(item_num)
            
            train_items[item_num] = [] 
            train_items[item_num].append(sess_temp)
            attr1_item_opin[item_num] = []
            attr2_item_opin[item_num] = []
            attr3_item_opin[item_num] = []
            attr4_item_opin[item_num] = []
            attr5_item_opin[item_num] = []
            attr6_item_opin[item_num] = []
            attr7_item_opin[item_num] = []
            
            attr1_item_opin[item_num].append(a1)
            attr2_item_opin[item_num].append(a2)
            attr3_item_opin[item_num].append(a3)
            attr4_item_opin[item_num].append(a4)
            attr5_item_opin[item_num].append(a5)
            attr6_item_opin[item_num].append(a6)
            attr7_item_opin[item_num].append(a7)
            if meta_dataset == 'Beauty' or meta_dataset == 'Home_and_Kitchen':
                attr8_item_opin[item_num] = []
                attr9_item_opin[item_num] = []
                attr8_item_opin[item_num].append(a8)
                attr9_item_opin[item_num].append(a9)
            
            item_num += 1
    train_sess[sess_temp] = new_seq
    tra[0].append(sess_temp)
#     预测时考虑last item
    tra[1].append(new_seq[:-1])
    tra[2].append(new_seq[-1])
    
item_count =  item_num-1
user_count = len(tra[0])

fre_item_mat = np.zeros((item_count, item_count), dtype=int)
# 全部的共现关系Original 
for sess in train_sess.keys():
    item_seq = np.unique(train_sess[sess])
    for it1 in item_seq:
        for it2 in item_seq:
            if it1 != it2:
#                 print(sess)
                fre_item_mat[int(it1)-1, int(it2)-1] +=1 
                
fre_user_mat = np.zeros((user_count, user_count), dtype=int)
# 全部的共现关系Original 
for item_t in train_items.keys():
    user_seq = np.unique(train_items[item_t])
    for it1 in user_seq:
        for it2 in user_seq:
            if it1 != it2:
                fre_user_mat[int(it1)-1, int(it2)-1] +=1 

# building user_fre & item_fre csr_matrix
def spar_graph(fre_mat):
    indices = [] # 非0元素的列坐标，item_id -1
    all_data = [] # 非0元素值，opin_id
    indptr = [0] # i i-1 行非0元素个数，0-0， n-总共元素个数，行数+1
    for i_list in fre_mat:
        i_temp_list = np.nonzero(i_list)[0].tolist()
#         if len(i_temp_list) == 0:
#             non_co += 1
        indices += i_temp_list
        all_data += [1]*len(i_temp_list)
        indptr.append(indptr[-1]+len(i_temp_list))
#     print('item without co-occurren: ',str(non_co))
    # indptr:session长度累加和; indices:item_id 减1, 由每个session内item组成; data:item在session内的权重，全部为1.
    coo_mat = (all_data, indices, indptr)
    return coo_mat

item_co_mat = spar_graph(fre_item_mat)
user_co_mat = spar_graph(fre_user_mat)


# test_set [[u_id ...], [x_t-1...], [label...]
tes = [[],[],[]]
tes_seq_num = 0
for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    if all_seqs[-1] in item_dict:
        tes_seq_num += 1
        new_seq = []
        for x in all_seqs:
            new_seq.append(item_dict[x])
        tes[0].append(sess_temp)
    #     预测时考虑last item
        tes[1].append(new_seq[:-1])
        tes[2].append(item_dict[all_seqs[-1]])
    

print('item_num: ',str(item_count))
print('train_seq_num & user_num: ',str(user_count))
print('test_seq_num: ',str(tes_seq_num))

inter_count += tes_seq_num

def spar_matrix(tra_seq):
    indices = [] # 非0元素的列坐标，item_id -1
    all_data = [] # 非0元素值，opin_id
    indptr = [0] # i i-1 行非0元素个数，0-0， n-总共元素个数，行数+1
    for sess_temp in tra_seq.keys():
        it_list = tra_seq[sess_temp]
        it_index = np.unique(it_list) - 1
        indices += (it_index).tolist()
        all_data += [1]*len(it_index)
        indptr.append(indptr[-1] + len(it_index.tolist()))
    results = (all_data, indices, indptr)
    return results

ui_co_mat = spar_matrix(train_sess)




# 返回 [[u1,u2],[attr1_opin, attr1_opin]]
def formulate_seq(dict_ui, dict_op):
    results = [[],[]]
    temp_res = [[],[]]
    max_len =0
    for x in dict_ui.keys():
        seqs_id = dict_ui[x]
        opin_id = dict_op[x]
        new_us_seq = []
        new_op_seq = []
        for id_t, op_t in zip(seqs_id, opin_id):
            if op_t != 0 and id_t not in new_us_seq:
                new_us_seq.append(id_t)
                new_op_seq.append(op_t)
        if len(new_us_seq)==0:
            new_us_seq.append(0)
            new_op_seq.append(0)
        if len(new_us_seq)>max_len:
            max_len = len(new_us_seq)
        temp_res[0].append(new_us_seq)
        temp_res[1].append(new_op_seq)
    for ui_seq, op_seq in zip(temp_res[0],temp_res[1]):
        pad_len = max_len - len(ui_seq)
        pad_ui_seq = ui_seq + [0]*pad_len
        pad_op_seq = op_seq + [0]*pad_len
        results[0].append(pad_ui_seq)
        results[1].append(pad_op_seq)
#     print('max_len:',str(max_len))
    return results

# u_seq = {1:[1,2,3,1],2:[4,5], 3:[7,8,9]}
# u_op = {1:[45,56,0,45],2:[66,0], 3:[0,33,0]}
# # [[[1,2],[4],[8]],[[45,56],[66],[33]]]
# formulate_seq(u_seq,u_op)

ui_1 = formulate_seq(train_sess,attr1_all)
ui_2 = formulate_seq(train_sess,attr2_all)
ui_3 = formulate_seq(train_sess,attr3_all)
ui_4 = formulate_seq(train_sess,attr4_all)
ui_5 = formulate_seq(train_sess,attr5_all)
ui_6 = formulate_seq(train_sess,attr6_all)
ui_7 = formulate_seq(train_sess,attr7_all)
iu_1 = formulate_seq(train_items,attr1_item_opin)
iu_2 = formulate_seq(train_items,attr2_item_opin)
iu_3 = formulate_seq(train_items,attr3_item_opin)
iu_4 = formulate_seq(train_items,attr4_item_opin)
iu_5 = formulate_seq(train_items,attr5_item_opin)
iu_6 = formulate_seq(train_items,attr6_item_opin)
iu_7 = formulate_seq(train_items,attr7_item_opin)

if meta_dataset == 'Beauty' or meta_dataset == 'Home_and_Kitchen':
    ui_8 = formulate_seq(train_sess,attr8_all)
    ui_9 = formulate_seq(train_sess,attr9_all)
    iu_8 = formulate_seq(train_items,attr8_item_opin)
    iu_9 = formulate_seq(train_items,attr9_item_opin)

def ui_seq(dict_ui):
    results = []
    temp_res = []
    max_len =0
    for x in dict_ui.keys():
        seqs_id = dict_ui[x]
        if len(seqs_id)>max_len:
            max_len = len(seqs_id)
        temp_res.append(seqs_id)
    for temp_l in temp_res:
        pad_len = max_len - len(temp_l)
        pad_ui_seq = temp_l + [0]*pad_len
        results.append(pad_ui_seq)
    return results

ui_list = ui_seq(train_sess)
iu_list = ui_seq(train_items)

print('#interactions: ',inter_count)

print('sequence average length: ', (inter_count)/(user_count) * 1.0)
# print('cold sequence average length: ', (all_train+all_tes_nocold+all_tes_cold)/(len(tra_seqs) + len(tes_seqs_nocold) +len(tes_seqs_cold) * 1.0))
train_data_path = './datasets/' + meta_dataset

if not os.path.exists(train_data_path):
    os.makedirs(train_data_path)
path_data_train = train_data_path + "/train.txt"
path_normal_test = train_data_path + "/test.txt"

tra_save = (tra[0], tra[1], tra[2],item_co_mat,user_co_mat, ui_list, iu_list, ui_1, ui_2, ui_3, ui_4, ui_5, ui_6, ui_7, iu_1, iu_2, iu_3, iu_4, iu_5, iu_6, iu_7)
tes_save = (tes[0], tes[1], tes[2])

if meta_dataset == 'Beauty' or meta_dataset == 'Home_and_Kitchen':
    tra_save = (tra[0], tra[1], tra[2],item_co_mat,user_co_mat, ui_list, iu_list, ui_1, ui_2, ui_3, ui_4, ui_5, ui_6, ui_7,ui_8, ui_9, iu_1, iu_2, iu_3, iu_4, iu_5, iu_6, iu_7, iu_8, iu_9)

pickle.dump(tra_save, open(path_data_train, 'wb'))
pickle.dump(tes_save, open(path_normal_test, 'wb'))
print("dataset: ", meta_dataset)
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
print("done")

item_num:  12391
train_seq_num & user_num:  12373
test_seq_num:  12311
#interactions:  110313
sequence average length:  8.915622726905358
dataset:  Yelp
2023-12-29 08:51:17
done
